# Análisis de MNIST con eager execution, tf.keras y tf.data

**Profesor:** Roberto Muñoz <br />
**E-mail:** <rmunoz@metricarts.com> <br />

**Profesor:** Sebastián Arpón <br />
**E-mail:** <rmunoz@metricarts.com> <br />

En este laboratorio crearemos una red neuronal que pueda detectar a que digito corresponde una imagen que recibe (note que cada imagen contendra solo un digito). Utilizaremos la API `tf.data` [Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) la cual es muy eficiente e incluye funcionalidades como el shuffling y batching. 

El conjunto de datos con el que trabajaremos es el MINST el cual, como veremos mas adelante esta incluido en KERAS.

In [ ]:
import tensorflow as tf

# Activando Eager
tf.enable_eager_execution()

import os
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
tf.__version__

## Usamos la API de keras para descargar el dataset de MNIST

In [ ]:
# obteniendo la data
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

In [ ]:
print("Tamaño train dataset: ", len(train_labels))

np.unique(train_labels)

In [ ]:
print("Tamaño test dataset: ", len(test_labels))

np.unique(test_labels)

## Revisamos el tamaño de train_images y train_labels

In [ ]:
print("Train images")
print("Tipo: ", type(train_images))
print("Nº de elementos: ", len(train_images))
print("Dimensiones: ", train_images.shape)

In [ ]:
print("Train labels")
print("Tipo: ", type(train_labels))
print("Nº de elementos: ", len(train_labels))
print("Dimensiones: ", train_labels.shape)

In [ ]:
train_labels[0:5]

In [ ]:
plt.imshow(train_images[4,:,:])

## Revisamos un par de imágenes del dataset train

In [ ]:
i=np.random.randint(len(train_images))

print("Indice del registro: ", i)
print("Label: ", train_labels[i])
print("Tamaño en pixels: ", train_images[i].shape)
plt.imshow(train_images[i,:,:])

## Transformamos train_images y test_images

Transformamos train_images y test_images de matrices de 28x28 a un vector de largo 784

In [ ]:
# Chequeando el tamaño de los conjuntos de entrenamiento y test
TRAINING_SIZE = len(train_images)
TEST_SIZE = len(test_images)

# transformando desde (N, 28, 28) a (N, 784)
train_images = np.reshape(train_images, (TRAINING_SIZE, 784))
test_images = np.reshape(test_images, (TEST_SIZE, 784))

# Transformando cada arreglo desde uint8 a float32
train_images = train_images.astype(np.float32)
test_images = test_images.astype(np.float32)

# Convirtiendo cada valor desde [0,255] a [0,1] 
train_images /= 255
test_images /=  255

In [ ]:
print("Train images")
print(len(train_images))
print(train_images.shape)

In [ ]:
print("Test images")
print(len(test_images))
print(test_images.shape)

## Transformamos las etiquetas en un vector 

Usamos la función **tf.keras.utils.to_categorical** del módulo **tf.keras** para transformar cada valor de etiqueta a un vector conformato categórico.

En el caso de usar la función de costo **categorical_crossentropy**, las etiquetas deben ser transformadas en formato categórico. Si tenemos 10 clases, entonces cada etiqueta debe ser transformado en un vector de largo 10 donde **todos los valores son cero** excepto el índice correspondiente a la clase el cual tendrá el **valor uno**.

In [ ]:
if 'train_labels_ORIG' not in locals():
    train_labels_ORIG=train_labels.copy()
    
if 'test_labels_ORIG' not in locals():
    test_labels_ORIG=test_labels.copy()

In [ ]:
NUM_DIGITS = 10

train_labels  = tf.keras.utils.to_categorical(train_labels_ORIG, NUM_DIGITS)
test_labels = tf.keras.utils.to_categorical(test_labels_ORIG, NUM_DIGITS)

print("Previo al cambio de formato\t: ", train_labels_ORIG[0]) # The format of the labels before conversion
print("Posterior al cambio de formato\t: ", train_labels[0]) # The format of the labels after conversion

In [ ]:
print(type(train_labels))
print(train_labels.dtype)

In [ ]:
train_labels[0]

In [ ]:
train_labels = train_labels.astype(np.float32)
test_labels = test_labels.astype(np.float32)

In [ ]:
print(type(train_labels))
print(train_labels.dtype)

In [ ]:
train_labels[0]

## Definimos la arquitectura de la red neuronal con Keras

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(300, activation=tf.nn.relu, input_shape=(784,)))
model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))

## Definimos el optimizador que usaremos

Esto es obligatorio mientras usamos eager execution

Usaremos el optimizador RMS Propagation

Más info en https://www.tensorflow.org/api_guides/python/train#Optimizers

In [ ]:
optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001)
#optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)

## Elegimos la función de costo y compilamos el modelo

Usaremos la función de costo categorical_crossentropy

Más info de losses en https://www.tensorflow.org/api_docs/python/tf/keras/losses

Más info de metrics en https://www.tensorflow.org/api_docs/python/tf/keras/metrics

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

model.summary()

## 1. Entrenamos el modelo usando tf.data y train_on_batch()

### Paso 1 - Creamos un dataset del tipo tf.data

Ahora usaremos `tf.data.Dataset` [API](https://www.tensorflow.org/api_docs/python/tf/data) para convertir los arreglos de Numpy en un dataset de TensorFlow

A continuacion crearemos un ciclo **for** que servira como una introduccion en la creacion de ciclos de entrenamientos personalizados. Aunque esencialmente estos ciclos hacer lo mismo que `model.fit`, esto nos permite personalizar todo el proceso y recolectar distintas metricas.

Usamos un batch size de 128 elementos

In [ ]:
BATCH_SIZE=128

# Dado que tf.data puede funcionar en colecciones de datos potencialmente grandes
# La desordenaremos por partes.
SHUFFLE_SIZE = 10000 

# Creando el dataset
dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
dataset = dataset.shuffle(SHUFFLE_SIZE)
dataset = dataset.batch(BATCH_SIZE)

### Step 2 - Definimos las epocas de entrenamiento y entrenamos

Aca entrenaremos sobre el dataset usando los distintos batch.

In [ ]:
EPOCHS=5

for epoch in range(EPOCHS):
    for images, labels in dataset:
        train_loss, train_accuracy = model.train_on_batch(images, labels)
  
  # Obtenemos cualquier metrica o ajustamos los parametros de entrenamiento
    print('Epoch #%d\t Loss: %.6f\tAccuracy: %.6f' % (epoch + 1, train_loss, train_accuracy))
  

In [ ]:
loss, accuracy = model.evaluate(test_images, test_labels)
print('Test accuracy: %.2f' % (accuracy))

## 2. Entrenamos el modelo usando fit()

### Paso 1 - Creamos una función que define el modelo

Usaremos **sparse_categorical_crossentropy**

In [ ]:
# Returns a short sequential model
def create_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(784,)))
    model.add(tf.keras.layers.Dropout(rate=0.8))
    model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))
  
    model.compile(optimizer=tf.train.AdamOptimizer(),
                  loss=tf.keras.losses.sparse_categorical_crossentropy,
                  metrics=['accuracy'])
  
    return model


# Create a basic model instance
model = create_model()
model.summary()

### Paso 2 - Entrenamos el modelo usando fit()

Dado que usaremos la función de costo **tf.keras.losses.sparse_categorical_crossentropy**, las etiquetas deben ser valores simples y no vectores

In [ ]:
train_labels = train_labels_ORIG.copy()
test_labels = test_labels_ORIG.copy()

In [ ]:
train_labels

In [ ]:
EPOCHS=5
checkpoint_dir = "results"

if not os.path.exists(checkpoint_dir):
    os.mkdir(checkpoint_dir)

In [ ]:
checkpoint_path = os.path.join(checkpoint_dir, "model_mnist_{epoch:04d}.ckpt")

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
                checkpoint_path, verbose=1, save_weights_only=True,
                # Save weights, every 1-epochs
                period=1)
                                                 

model = create_model()

model.fit(train_images, train_labels,  epochs = EPOCHS, 
          validation_data = (test_images, test_labels),
          callbacks = [cp_callback])  # pass callback to training

## Evaluamos el modelo recién entrenado

In [ ]:
loss, acc = model.evaluate(test_images, test_labels)

print("Model accuracy: {:5.2f}%".format(100*acc))

## Restauramos una época del modelo

In [ ]:
print(checkpoint_dir)

In [ ]:
import glob

pattern = os.path.join(checkpoint_dir, "*.ckpt.data*")
checkpoints = sorted(glob.glob(pattern))

for file in checkpoints:
    print(file)

In [ ]:
weight_file = 'results/model_mnist_0001.ckpt'

model = create_model()
model.load_weights(weight_file)
loss, acc = model.evaluate(test_images, test_labels)

print("Restored model - accuracy: {:5.2f}%".format(100*acc))